In [ ]:
from IPython.display import Markdown as md
import os
import pandas as pd

# Some Randomness, for Fun

In [ ]:
import random
random.seed(0) # pick your seed

# Data

In [ ]:
!pip install eurostatapiclient

In [ ]:
from eurostatapiclient import EurostatAPIClient

#Set versions and formats, so far only the ones used here are availeable and call client
VERSION = 'v2.1'
FORMAT = 'json'
LANGUAGE = 'en'
client = EurostatAPIClient(VERSION, FORMAT, LANGUAGE)

In [ ]:
%%html
<iframe src="https://ec.europa.eu/eurostat/databrowser/view/env_waspacr/default/table?lang=en" width="1000" height="800"></iframe>

In [ ]:
countries_names = {'AT':'Austria', 'BE':'Belgium', 'BG':'Bulgaria', 'CY': 'Cyprus', 
                   'CZ': 'Czechia', 'DE': 'Germany', 'DK': 'Denmark', 'EE':'Estonia', 
                   'EL': 'Greece', 'ES':'Spain', 'FI':'Finland', 'FR':'France', 
                   'HR':'Croatia', 'HU':'Hungary', 'IE':'Ireland', 'IT':'Italy', 
                   'LT':'Lithuania', 'LU':'Luxembourg', 'LV':'Latvia', 'MT': 'Malta', 
                   'NL':'Netherlands', 'PL':'Poland', 'PT':'Portugal', 'RO':'Romania', 
                   'SE':'Sweden', 'SI':'Slovenia', 'SK':'Slovakia', 'UK':'United Kingdom'}

packaging_types = {'W150101': 'Paper and cardboard packaging',
                   'W150102': 'Plastic packaging',
                   'W150103': 'Wooden packaging',
                   'W150104': 'Metallic packaging',
                   'W150107': 'Glass packaging',
                   'W150199': 'Other packaging'}

In [ ]:
from requests import get

In [ ]:
'''
response = get('http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/pat_ep_nipc?precision=1&unit=NR&ipc=A')
print(response.status_code)
data_json = response.json()
'''

"\nresponse = get('http://ec.europa.eu/eurostat/wdds/rest/data/v2.1/json/en/pat_ep_nipc?precision=1&unit=NR&ipc=A')\nprint(response.status_code)\ndata_json = response.json()\n"

In [ ]:
par_df1 = {
    'waste': packaging_types.keys(),
    'unit': ['RT'],
    'geo': list(countries_names.keys()),
}

df1 = client.get_dataset('env_waspacr', params=par_df1).to_dataframe()

df1.rename(columns={'geo': 'country', 'time': 'year'}, inplace=True)
df1['year'] = df1['year'].astype('int')
df1['country'] = df1['country'].map(countries_names)
df1['waste'] = df1['waste'].map(packaging_types)

In [ ]:
print(len(df1))
print(df1.dtypes)
df1.sample(5)

3696
values     float64
waste       object
unit        object
country     object
year         int64
dtype: object


,values,waste,unit,country,year
2054,59.7,Metallic packaging,RT,Spain,2005
958,33.5,Plastic packaging,RT,Italy,2009
171,74.2,Paper and cardboard packaging,RT,Estonia,2014
2834,74.3,Glass packaging,RT,Lithuania,2015
2198,72.0,Metallic packaging,RT,Italy,2017


In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive', force_remount=True)
    dir = os.path.join('gdrive', 'MyDrive', 'Eurostat', '01 - Intro to Python for Data Science')
else:
    dir = '.'
    
data_dir = os.path.join(dir, 'data')
os.makedirs(data_dir, exist_ok=True)
data_dir

Mounted at /content/gdrive


'gdrive/MyDrive/Eurostat/01 - Intro to Python for Data Science/data'

In [ ]:
filename = os.path.join(data_dir, 'env_waspacr.csv')
df1.to_csv(filename, index=False)

In [ ]:
df2 = df1.pivot(index='year', columns=['waste', 'country'], values='values')

In [ ]:
df2

waste   Paper and cardboard packaging          ... Other packaging               
country                       Austria Belgium  ...        Slovakia United Kingdom
year                                           ...                               
1997                             85.0    77.5  ...             NaN            0.0
1998                             84.3    83.1  ...             NaN            0.0
1999                             87.7    69.7  ...             NaN            0.0
2000                             86.9    82.1  ...             NaN            0.0
2001                             81.4    85.8  ...             NaN            0.0
2002                             80.0    78.4  ...             NaN            0.0
2003                             81.5    79.2  ...             0.4            0.0
2004                             83.2    83.4  ...             NaN            0.0
2005                             86.4    83.3  ...             NaN            0.0
2006                             87.0    89.1  ...             NaN            0.0
2007                             83.5    92.0  ...             NaN            0.0
2008                             85.4    89.4  ...             NaN            0.0
2009                             84.8    88.0  ...            11.7            NaN
2010                             84.5    89.8  ...             2.0            0.0
2011                             84.5    90.4  ...             0.0            0.0
2012                             84.9    89.8  ...            29.0            0.0
2013                             84.3    89.1  ...            33.2            0.0
2014                             84.9    90.6  ...            20.5            0.0
2015                             84.9    90.7  ...            11.9            0.0
2016                             84.9    89.4  ...            15.4            0.0
2017                             83.5    92.9  ...             8.5            0.0
2018                             84.2    95.6  ...            23.8            0.0

[22 rows x 168 columns]

# Classes

## Attributes

In [ ]:
class WasteContainer():

  next_id = 1001                            # class attribute

  def __init__(self, package_type):
    self.package_type = package_type        # instance attribute
    self.id = WasteContainer.next_id        # instance attribute
    WasteContainer.next_id += 1

In [ ]:
glass_container = WasteContainer('glass')
print(f"id: {glass_container.id}", f"package: {glass_container.package_type}", sep='\n')

id: 1001
package: glass


In [ ]:
metal_container = WasteContainer('metal')
print(f"id: {metal_container.id}", f"package: {metal_container.package_type}", sep='\n')

id: 1002
package: metal


In [ ]:
WasteContainer.next_id

1003

### Precedence

In [ ]:
WasteContainer.next_id is glass_container.next_id

True

In [ ]:
glass_container.next_id is metal_container.next_id

True

In [ ]:
glass_container.next_id = 1003
glass_container.next_id is WasteContainer.next_id

False

## Methods

In [ ]:
class WasteContainer():

  next_id = 1001

  def __init__(self, package_type):
    self.package_type = package_type
    self.id = WasteContainer.next_id
    WasteContainer.next_id += 1
    self.kg = 0                               # instance attribute

  def fill(self, weight_kg):                  # instance method
    self.kg += weight_kg

In [ ]:
glass_container = WasteContainer('glass')
print(f"id: {glass_container.id}", f"package: {glass_container.package_type}", f"weight: {glass_container.kg}", sep='\n')

id: 1001
package: glass
weight: 0


In [ ]:
glass_container.fill(2)
print(f"id: {glass_container.id}", f"package: {glass_container.package_type}", f"weight: {glass_container.kg}", sep='\n')

id: 1001
package: glass
weight: 2


In [ ]:
glass_container.fill(5)
print(f"id: {glass_container.id}", f"package: {glass_container.package_type}", f"weight: {glass_container.kg}", sep='\n')

id: 1001
package: glass
weight: 7


### ❓ Exercise

In [ ]:
md(f"##❓ Write a method _empty(self)_ that resets the container's weight to zero.")

##❓ Write a method _empty(self)_ that resets the container's weight to zero.

In [ ]:
class WasteContainer():

  next_id = 1001

  def __init__(self, package_type):
    self.package_type = package_type
    self.id = WasteContainer.next_id
    WasteContainer.next_id += 1
    self.kg = 0

  def fill(self, weight_kg):
    self.kg += weight_kg

  def empty(self):
    pass

In [ ]:
glass_container = WasteContainer('glass')
print("-- Feed container some waste")
glass_container.fill(5)
print(f"id: {glass_container.id}", f"package: {glass_container.package_type}", f"weight: {glass_container.kg}", sep='\n')
print("-- Empty container")
glass_container.empty()
print(f"id: {glass_container.id}", f"package: {glass_container.package_type}", f"weight: {glass_container.kg}", sep='\n')

-- Feed container some waste
id: 1001
package: glass
weight: 5
-- Empty container
id: 1001
package: glass
weight: 5


## Decorators

### @classmethod

In [ ]:
class WasteContainer():

  _next_id = 1001                             # _ indicates this variable is not meant for public twiddling

  @classmethod
  def _generate_id(cls):                      # class method
    id = cls._next_id
    cls._next_id += 1
    return id
  
  def __init__(self, package_type):
    self.package_type = package_type
    self.id = WasteContainer._generate_id()   # invocation of to class method
    self.kg = 0

  def fill(self, weight_kg):
    self.kg += weight_kg

  def empty(self):
    self.kg = 0

In [ ]:
glass_container = WasteContainer('glass')
print(f"id: {glass_container.id}", f"package: {glass_container.package_type}", f"weight: {glass_container.kg}", sep='\n')

id: 1001
package: glass
weight: 0


In [ ]:
metal_container = WasteContainer('metal')
metal_container.fill(100)
print(f"id: {metal_container.id}", f"package: {metal_container.package_type}", f"weight: {metal_container.kg}", sep='\n')

id: 1002
package: metal
weight: 100


### @property

In [ ]:
plastic_container = WasteContainer('plastic')
plastic_container.fill(100)
print(f"id: {plastic_container.id}", f"package: {plastic_container.package_type}", f"weight: {plastic_container.kg}", sep='\n')

id: 1003
package: plastic
weight: 100


In [ ]:
plastic_container.kg = 200
plastic_container.kg

200

In [ ]:
class WasteContainer():

  _next_id = 1001

  @classmethod
  def _generate_id(cls):
    id = cls._next_id
    cls._next_id += 1
    return id
  
  def __init__(self, package_type):
    self.package_type = package_type
    self.id = WasteContainer._generate_id()
    self._kg = 0

  @property
  def kg(self):
    return self._kg

  def fill(self, weight_kg):
    self._kg += weight_kg

  def empty(self):
    self._kg = 0

In [ ]:
plastic_container = WasteContainer('plastic')
plastic_container.fill(100)
print(f"id: {plastic_container.id}", f"package: {plastic_container.package_type}", f"weight: {plastic_container.kg}", sep='\n')
#plastic_container.kg = 200

id: 1001
package: plastic
weight: 100


### @property.setter

In [ ]:
class WasteContainer():

  _next_id = 1001

  @classmethod
  def _generate_id(cls):
    id = cls._next_id
    cls._next_id += 1
    return id
  
  def __init__(self, package_type):
    self.package_type = package_type
    self.id = WasteContainer._generate_id()
    self._kg = 0

  @property
  def kg(self):
    return self._kg

  @kg.setter
  def kg(self, weight_kg):
    self._kg = weight_kg

  def fill(self, weight_kg):
    self._kg += weight_kg

  def empty(self):
    self._kg = 0

In [ ]:
plastic_container = WasteContainer('plastic')
plastic_container.fill(100)
print(f"id: {plastic_container.id}", f"package: {plastic_container.package_type}", f"weight: {plastic_container.kg}", sep='\n')
plastic_container.kg = 200
plastic_container.kg

id: 1001
package: plastic
weight: 100


200

In [ ]:
class WasteContainer():

  _next_id = 1001

  @classmethod
  def _generate_id(cls):
    id = cls._next_id
    cls._next_id += 1
    return id
  
  def __init__(self, package_type):
    self.package_type = package_type
    self.id = WasteContainer._generate_id()
    self.kg = 0

  @property
  def kg(self):
    return self._kg

  @kg.setter
  def kg(self, weight_kg):
    if weight_kg > 1000:
      raise ValueError("Container too heavy")
    self._kg = weight_kg

  def fill(self, weight_kg):
    self.kg = self.kg + weight_kg

  def empty(self):
    self.kg = 0

In [ ]:
plastic_container = WasteContainer('plastic')
plastic_container.fill(100)
print(f"id: {plastic_container.id}", f"package: {plastic_container.package_type}", f"weight: {plastic_container.kg}", sep='\n')
plastic_container.kg = 200
plastic_container.kg

id: 1001
package: plastic
weight: 100


200

In [ ]:
metal_container = WasteContainer('plastic')
metal_container.fill(100)
print(f"id: {metal_container.id}", f"package: {metal_container.package_type}", f"weight: {metal_container.kg}", sep='\n')
#metal_container.fill(9500)

id: 1002
package: plastic
weight: 100


### @staticmethod

In [ ]:
class WasteContainer():

  _next_id = 1001

  @staticmethod
  def _kg_to_lbs(weight_kg):
    weight_lbs = weight_kg * 2.20462
    return weight_lbs

  @staticmethod
  def _lbs_to_kg(weight_lbs):
    weight_kg = weight_lbs / 2.20462
    return weight_kg

  @classmethod
  def _generate_id(cls):
    id = cls._next_id
    cls._next_id += 1
    return id
  
  def __init__(self, package_type):
    self.package_type = package_type
    self.id = WasteContainer._generate_id()
    self.kg = 0

  @property
  def kg(self):
    return self._kg

  @kg.setter
  def kg(self, weight_kg):
    if weight_kg > 1000:
      raise ValueError("Container too heavy")
    self._kg = weight_kg

  def fill(self, weight_kg):
    self.kg = self.kg + weight_kg

  def empty(self):
    self.kg = 0

### ❓ Exercise

In [ ]:
md(f"##❓ Write a property (and its setter) that allow to access and manipulate the container's weight in pounds (lbs). The weight in kg and lbs must be linked by the 1:2.20462 ratio at all times!")

##❓ Write a property (and its setter) that allow to access and manipulate the container's weight in pounds (lbs). The weight in kg and lbs must be linked by the 1:2.20462 ratio at all times!

In [ ]:
class WasteContainer():

  _next_id = 1001

  @staticmethod
  def _kg_to_lbs(weight_kg):
    weight_lbs = weight_kg * 2.20462
    return weight_lbs

  @staticmethod
  def _lbs_to_kg(weight_lbs):
    weight_kg = weight_lbs / 2.20462
    return weight_kg

  @classmethod
  def _generate_id(cls):
    id = cls._next_id
    cls._next_id += 1
    return id
  
  def __init__(self, package_type):
    self.package_type = package_type
    self.id = WasteContainer._generate_id()
    self.kg = 0

  @property
  def kg(self):
    return self._kg

  @kg.setter
  def kg(self, weight_kg):
    if weight_kg > 1000:
      raise ValueError("Container too heavy")
    self._kg = weight_kg

  # your code goes here

  def fill(self, weight_kg):
    self.kg = self.kg + weight_kg

  def empty(self):
    self.kg = 0

In [ ]:
paper_container = WasteContainer('paper')
paper_container.fill(100)
#print(f"id: {paper_container.id}", f"package: {paper_container.package_type}", f"weight: {paper_container.kg:.2f} kg, {paper_container.lbs:.2f} lbs", sep='\n')

In [ ]:
paper_container.lbs = 500
#print(f"id: {paper_container.id}", f"package: {paper_container.package_type}", f"weight: {paper_container.kg:.2f} kg, {paper_container.lbs:.2f} lbs", sep='\n')

# String Representations

## docstring

In [ ]:
help(WasteContainer)

Help on class WasteContainer in module __main__:

class WasteContainer(builtins.object)
 |  WasteContainer(package_type)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, package_type)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  empty(self)
 |  
 |  fill(self, weight_kg)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  kg



In [ ]:
class WasteContainer():
  """
  WasteContainers can be filled with package waste of a specific type and up to a maximum weight.
  """

  _next_id = 1001

  @staticmethod
  def _kg_to_lbs(weight_kg):
    weight_lbs = weight_kg * 2.20462
    return weight_lbs

  @staticmethod
  def _lbs_to_kg(weight_lbs):
    weight_kg = weight_lbs / 2.20462
    return weight_kg

  @classmethod
  def _generate_id(cls):
    id = cls._next_id
    cls._next_id += 1
    return id
  
  def __init__(self, package_type):
    """
    The type of package waste is specified at initialization.
    """
    self.package_type = package_type
    self.id = WasteContainer._generate_id()
    self.kg = 0

  @property
  def kg(self):
    """
    The weight of waste, in kilogramms.
    """
    return self._kg

  @kg.setter
  def kg(self, weight_kg):
    if weight_kg > 1000:
      raise ValueError("Container too heavy")
    self._kg = weight_kg

  @property
  def lbs(self):
    """
    The weight of waste, in pounds.
    """
    return WasteContainer._kg_to_lbs(self.kg)

  @lbs.setter
  def lbs(self, weight_lbs):
    self.kg = WasteContainer._lbs_to_kg(weight_lbs)

  def fill(self, weight_kg):
    '''
    Add weight_kg kilograms of waste to the container.
    '''
    self.kg = self.kg + weight_kg

  def empty(self):
    '''
    Emptying the container resets its weight to zero.
    '''
    self.kg = 0

In [ ]:
help(WasteContainer)

Help on class WasteContainer in module __main__:

class WasteContainer(builtins.object)
 |  WasteContainer(package_type)
 |  
 |  WasteContainers can be filled with package waste of a specific type and up to a maximum weight.
 |  
 |  Methods defined here:
 |  
 |  __init__(self, package_type)
 |      The type of package waste is specified at initialization.
 |  
 |  empty(self)
 |      Emptying the container resets its weight to zero.
 |  
 |  fill(self, weight_kg)
 |      Add weight_kg kilograms of waste to the container.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of weak references to the object (if defined)
 |  
 |  kg
 |      The weight of waste, in kilogramms.
 |  
 |  lbs
 |      The weight of waste, in pounds.



## repr

In [ ]:
paper_container = WasteContainer('paper')
paper_container.fill(100)
paper_container

In [ ]:
class PrintableWasteContainer(WasteContainer):
  def __repr__(self):
    return f"WasteContainer(id: {self.id}, package_type: {self.package_type}, kg: {self.kg})"

In [ ]:
paper_container = PrintableWasteContainer('paper')
paper_container.fill(100)
paper_container

WasteContainer(id: 1002, package_type: paper, kg: 100)

## str

In [ ]:
paper_container = PrintableWasteContainer('paper')
paper_container.fill(950)
print(paper_container, 'needs to be emptied soon')

WasteContainer(id: 1003, package_type: paper, kg: 950) needs to be emptied soon


In [ ]:
class PrintableWasteContainer(WasteContainer):
  def __repr__(self):
    return f"WasteContainer(id: {self.id}, package_type: {self.package_type}, kg: {self.kg})"

  def __str__(self):
    return f"Container #{self.id} holding {self.kg:.0f} kg of {self.package_type}"

In [ ]:
paper_container = PrintableWasteContainer('paper')
paper_container.fill(950)
print(paper_container, 'needs to be emptied soon')

Container #1004 holding 950 kg of paper needs to be emptied soon


## format

In [ ]:
class PrintableWasteContainer(WasteContainer):
  def __repr__(self):
    return f"WasteContainer(id: {self.id}, package_type: {self.package_type}, kg: {self.kg})"

  def __str__(self):
    return f"Container #{self.id} holding {self.kg:.0f} kg of {self.package_type}"

  def __format__(self, format_spec):
    if format_spec == 'lbs':
      return f"Container #{self.id} holding {self.lbs:.0f} lbs of {self.package_type}"
    else:
      return f"Container #{self.id} holding {self.kg:.0f} kg of {self.package_type}"

In [ ]:
paper_container = PrintableWasteContainer('paper')
paper_container.fill(950)
print(f"{paper_container} needs to be emptied soon")
print(f"{paper_container:lbs} needs to be emptied soon")

Container #1005 holding 950 kg of paper needs to be emptied soon
Container #1005 holding 2094 lbs of paper needs to be emptied soon


# Inheritance

In [ ]:
help(PrintableWasteContainer)

Help on class PrintableWasteContainer in module __main__:

class PrintableWasteContainer(WasteContainer)
 |  PrintableWasteContainer(package_type)
 |  
 |  WasteContainers can be filled with package waste of a specific type and up to a maximum weight.
 |  
 |  Method resolution order:
 |      PrintableWasteContainer
 |      WasteContainer
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __format__(self, format_spec)
 |      Default object formatter.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  __str__(self)
 |      Return str(self).
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from WasteContainer:
 |  
 |  __init__(self, package_type)
 |      The type of package waste is specified at initialization.
 |  
 |  empty(self)
 |      Emptying the container resets its weight to zero.
 |  
 |  fill(self, weight_kg)
 |      Add weight_kg kilograms of waste to the container.
 |  
 |  --------------------------

In [ ]:
class GlassContainer(PrintableWasteContainer):

  def __init__(self, glass_color):
    self._glass_color = glass_color
    super().__init__('glass')

  @property
  def glass_color(self):
    return self._glass_color

In [ ]:
glass_container = GlassContainer('green')
glass_container

WasteContainer(id: 1006, package_type: glass, kg: 0)

In [ ]:
type(glass_container).__name__

'GlassContainer'

In [ ]:
class PrintableWasteContainer(WasteContainer):
  def __repr__(self):
    #return f"{WasteContainer}(id: {self.id}, package_type: {self.package_type}, kg: {self.kg})"
    return f"{type(self).__name__}(id: {self.id}, package_type: {self.package_type}, kg: {self.kg})"

  def __str__(self):
    return f"Container #{self.id} holding {self.kg:.0f} kg of {self.package_type}"

  def __format__(self, format_spec):
    if format_spec == 'lbs':
      return f"Container #{self.id} holding {self.lbs:.0f} lbs of {self.package_type}"
    else:
      return f"Container #{self.id} holding {self.kg:.0f} kg of {self.package_type}"

In [ ]:
glass_container = GlassContainer('green')
glass_container

WasteContainer(id: 1007, package_type: glass, kg: 0)

In [ ]:
waste_container = PrintableWasteContainer('glass')
waste_container

PrintableWasteContainer(id: 1008, package_type: glass, kg: 0)

In [ ]:
print(glass_container.glass_color)
#print(waste_container.glass_color)

green


### ❓ Exercise

In [ ]:
md(f"##❓ Override the __str__() function of GlassContainer, to indicate the color of the glass!")

##❓ Override the __str__() function of GlassContainer, to indicate the color of the glass!